# 导入必要的库

In [1]:
import mxnet as mx
from mxnet import gluon
from mxnet import ndarray as nd
from mxnet import autograd
from mxnet.gluon import nn

import h5py
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

ctx = mx.gpu()

In [2]:
def accuracy(output, labels):
    return nd.mean(nd.argmax(output, axis=1) == labels).asscalar()

def evaluate(net, data_iter):
    loss, acc, n = 0., 0., 0.
    steps = len(data_iter)
    for data, label in data_iter:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        output = net(data)
        acc += accuracy(output, label)
        loss += nd.mean(softmax_cross_entropy(output, label)).asscalar()
    return loss/steps, acc/steps

# 载入之前计算的特征向量

In [3]:
with h5py.File('features.h5', 'r') as f:
    features_vgg = np.array(f['vgg'])
    features_resnet = np.array(f['resnet'])
    features_densenet = np.array(f['densenet'])
    features_inception = np.array(f['inception'])
    labels = np.array(f['labels'])

In [4]:
features_resnet = features_resnet.reshape(features_resnet.shape[:2])
features_inception = features_inception.reshape(features_inception.shape[:2])

features = np.concatenate([features_resnet, features_densenet, features_inception], axis=-1)

# 利用 ArrayDataset 和 DataLoader 构建迭代器

In [5]:
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2)

dataset_train = gluon.data.ArrayDataset(nd.array(X_train), nd.array(y_train))
dataset_val = gluon.data.ArrayDataset(nd.array(X_val), nd.array(y_val))

batch_size = 128
data_iter_train = gluon.data.DataLoader(dataset_train, batch_size, shuffle=True)
data_iter_val = gluon.data.DataLoader(dataset_val, batch_size)

# 构建模型

In [6]:
net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(256, activation='relu'))
    net.add(nn.Dropout(0.5))
    net.add(nn.Dense(120))

net.initialize(ctx=ctx)
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': 1e-4, 'wd': 1e-5})
net

Sequential(
  (0): Dense(None -> 256, Activation(relu))
  (1): Dropout(p = 0.5)
  (2): Dense(None -> 120, linear)
)

# 训练50代

In [7]:
for epoch in range(50):
    train_loss = 0.
    train_acc = 0.
    steps = len(data_iter_train)
    for data, label in data_iter_train:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        
        loss.backward()
        trainer.step(batch_size)
        
        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)
    
    val_loss, val_acc = evaluate(net, data_iter_val)
    print("Epoch %d. loss: %.4f, acc: %.2f%%, val_loss %.4f, val_acc %.2f%%" % (
        epoch+1, train_loss/steps, train_acc/steps*100, val_loss, val_acc*100))

Epoch 1. loss: 4.5991, acc: 5.27%, val_loss 3.8873, val_acc 26.60%
Epoch 2. loss: 3.4482, acc: 24.89%, val_loss 2.3933, val_acc 66.30%
Epoch 3. loss: 2.2899, acc: 49.16%, val_loss 1.3686, val_acc 80.34%
Epoch 4. loss: 1.5658, acc: 63.39%, val_loss 0.8952, val_acc 84.36%
Epoch 5. loss: 1.1688, acc: 71.79%, val_loss 0.6645, val_acc 86.07%
Epoch 6. loss: 0.9619, acc: 75.25%, val_loss 0.5501, val_acc 87.53%
Epoch 7. loss: 0.8236, acc: 77.80%, val_loss 0.4851, val_acc 87.97%
Epoch 8. loss: 0.7274, acc: 80.09%, val_loss 0.4418, val_acc 88.31%
Epoch 9. loss: 0.6635, acc: 82.39%, val_loss 0.4173, val_acc 88.12%
Epoch 10. loss: 0.6234, acc: 83.12%, val_loss 0.3868, val_acc 89.00%
Epoch 11. loss: 0.5592, acc: 84.63%, val_loss 0.3676, val_acc 89.00%
Epoch 12. loss: 0.5104, acc: 85.71%, val_loss 0.3538, val_acc 89.44%
Epoch 13. loss: 0.4838, acc: 86.50%, val_loss 0.3492, val_acc 89.20%
Epoch 14. loss: 0.4568, acc: 87.08%, val_loss 0.3333, val_acc 89.49%
Epoch 15. loss: 0.4347, acc: 87.71%, val_los

# 载入测试集特征向量

In [8]:
with h5py.File('features_test.h5', 'r') as f:
    features_vgg_test = np.array(f['vgg'])
    features_resnet_test = np.array(f['resnet'])
    features_densenet_test = np.array(f['densenet'])
    features_inception_test = np.array(f['inception'])

In [9]:
features_resnet_test = features_resnet_test.reshape(features_resnet_test.shape[:2])
features_inception_test = features_inception_test.reshape(features_inception_test.shape[:2])

features_test = np.concatenate([features_resnet_test, features_densenet_test, features_inception_test], axis=-1)

# 利用模型进行预测并输出到 pred.csv

In [10]:
output = nd.softmax(net(nd.array(features_test).as_in_context(ctx))).asnumpy()

In [11]:
df = pd.read_csv('sample_submission.csv')

for i, c in enumerate(df.columns[1:]):
    df[c] = output[:,i]

df.to_csv('pred.csv', index=None)